# Chapter 22: Reader

In [1]:
boop = (*2)
doop = (+10)
bip = boop . doop
bip 3

26

In [2]:
bloop = fmap boop doop
-- same as bip

bloop 3

26

In [3]:
bbop = (+) <$> boop <*> doop
bbop 3

19

In [4]:
import Control.Applicative (liftA2)

duwop = liftA2 (+) boop doop
duwop 3

19

In [5]:
wutwop x = boop x + doop x
wutwop 3

19

In [6]:
boopDoop = do
    a <- boop
    b <- doop
    return (a + b)
    
boopDoop 3

19

---

In [7]:
import Data.Char (toUpper)

cap :: [Char] -> [Char]
cap xs = map toUpper xs

rev :: [Char] -> [Char]
rev xs = reverse xs

Line 3: Use String
Found:
[Char] -> [Char]
Why not:
String -> StringLine 4: Eta reduce
Found:
cap xs = map toUpper xs
Why not:
cap = map toUpperLine 6: Use String
Found:
[Char] -> [Char]
Why not:
String -> StringLine 7: Eta reduce
Found:
rev xs = reverse xs
Why not:
rev = reverse

In [8]:
composed :: [Char] -> [Char]
composed = rev . cap

fmapped :: [Char] -> [Char]
fmapped = fmap rev cap

composed "Julie"
fmapped "Chris"

Line 1: Use String
Found:
[Char] -> [Char]
Why not:
String -> StringLine 4: Use String
Found:
[Char] -> [Char]
Why not:
String -> String

"EILUJ"

"SIRHC"

In [9]:
tupled :: [Char] -> ([Char], [Char])
tupled = liftA2 (,) rev cap

tupled "Julie"

Line 1: Use String
Found:
[Char] -> ([Char], [Char])
Why not:
String -> (String, String)

("eiluJ","JULIE")

In [10]:
tupled' :: [Char] -> ([Char], [Char])
tupled' = do
    a <- rev
    b <- cap
    return (a, b)
    
tupled' "Julie"

Line 1: Use String
Found:
[Char] -> ([Char], [Char])
Why not:
String -> (String, String)

("eiluJ","JULIE")

In [11]:
:t (>>=)

(>>=) :: forall (m :: * -> *) a b. Monad m => m a -> (a -> m b) -> m b

In [12]:
:t (>>=) :: (c -> a) -> (a -> c -> b) -> (c -> b) 

(>>=) :: (c -> a) -> (a -> c -> b) -> (c -> b) :: forall c a b. (c -> a) -> (a -> c -> b) -> c -> b

In [13]:
:t rev >>= (,)

rev >>= (,) :: [Char] -> ([Char], [Char])

In [14]:
:t cap >>= (,)

cap >>= (,) :: [Char] -> ([Char], [Char])

In [15]:
:t (,)

(,) :: forall a b. a -> b -> (a, b)

In [16]:
:t return

return :: forall (m :: * -> *) a. Monad m => a -> m a

In [17]:
:t return :: a -> c -> a

return :: a -> c -> a :: forall a c. a -> c -> a

In [18]:
tupled'' :: [Char] -> ([Char], [Char])
tupled'' = rev >>= (\a -> cap >>= (\b -> return (a, b)))

Line 1: Use String
Found:
[Char] -> ([Char], [Char])
Why not:
String -> (String, String)

In [19]:
tupled'' "Julie"

("eiluJ","JULIE")

In [20]:
tupled''' :: [Char] -> ([Char], [Char])
tupled''' = rev >>= (\a b -> (a, cap b))

Line 1: Use String
Found:
[Char] -> ([Char], [Char])
Why not:
String -> (String, String)

In [21]:
tupled''' "Julie"

("eiluJ","JULIE")

---

In [22]:
fmap (+1) (*2) 3

7

In [23]:
fmap (+1) (*2) $ 3

Line 1: Redundant $
Found:
fmap (+ 1) (* 2) $ 3
Why not:
fmap (+ 1) (* 2) 3

7

In [24]:
(fmap (+1) (*2)) 3

Line 1: Redundant bracket
Found:
(fmap (+ 1) (* 2)) 3
Why not:
fmap (+ 1) (* 2) 3

7

In [25]:
(+1) . (*2) $ 3

7

In [26]:
(+2) . (*1) $ 2

4

In [27]:
fmap (+2) (*1) $ 2

Line 1: Redundant $
Found:
fmap (+ 2) (* 1) $ 2
Why not:
fmap (+ 2) (* 1) 2

4

In [28]:
(+2) `fmap` (*1) $ 2

4

---

In [29]:
newtype Reader r a = Reader { runReader :: r -> a }

instance Functor (Reader r) where
    fmap f (Reader ra) = Reader $ fmap f ra

In [30]:
ask' :: Reader a a
ask' = Reader id

In [31]:
:t runReader

runReader :: forall r a. Reader r a -> r -> a

---

In [32]:
newtype HumanName = HumanName String deriving (Eq, Show)

newtype DogName = DogName String deriving (Eq, Show)

newtype Address = Address String deriving (Eq, Show)

In [33]:
data Person = Person
    { humanName :: HumanName
    , dogName :: DogName
    , address :: Address
    } deriving (Eq, Show)

In [34]:
data Dog = Dog
    { dogsName :: DogName
    , dogsAddress :: Address
    } deriving (Eq, Show)

In [35]:
pers :: Person
pers = Person 
    (HumanName "Big Bird") 
    (DogName "Barkley")
    (Address "Sesame Street")

In [36]:
chris :: Person
chris = Person
    (HumanName "Chris Allen")
    (DogName "Papu")
    (Address "Austin")

In [37]:
getDog :: Person -> Dog
getDog p = Dog (dogName p) (address p)

getDog pers
getDog chris

Dog {dogsName = DogName "Barkley", dogsAddress = Address "Sesame Street"}

Dog {dogsName = DogName "Papu", dogsAddress = Address "Austin"}

In [38]:
getDogR :: Person -> Dog
getDogR = Dog <$> dogName <*> address

getDogR pers
getDogR chris

Dog {dogsName = DogName "Barkley", dogsAddress = Address "Sesame Street"}

Dog {dogsName = DogName "Papu", dogsAddress = Address "Austin"}

In [39]:
getDogR' :: Person -> Dog
getDogR' = liftA2 Dog dogName address

getDogR' pers
getDogR' chris

Dog {dogsName = DogName "Barkley", dogsAddress = Address "Sesame Street"}

Dog {dogsName = DogName "Papu", dogsAddress = Address "Austin"}

---

In [40]:
myLiftA2 :: Applicative f => (a -> b -> c) -> f a -> f b -> f c
myLiftA2 f ma mb = f <$> ma <*> mb

In [41]:
asks :: (r -> a) -> Reader r a
asks = Reader

In [42]:
import Data.Function (on)

instance Applicative (Reader r) where
    pure = Reader . pure
    --(Reader rf) <*> (Reader ra) = Reader $ \r -> rf r (ra r)
    (Reader rf) <*> (Reader ra) = Reader $ rf <*> ra
    
runReader ((+) <$> Reader (*2) <*> Reader (*3)) 3

15

---

In [43]:
foo :: (Functor f, Num a) => f a -> f a
foo r = fmap (+1) r

foo [1, 2, 3]

Line 2: Eta reduce
Found:
foo r = fmap (+ 1) r
Why not:
foo = fmap (+ 1)

[2,3,4]

In [44]:
bar :: Foldable f => t -> f a -> (t, Int)
bar r t = (r, length t)

bar [2, 3, 4] [1, 2, 3]

([2,3,4],3)

In [45]:
froot :: Num a => [a] -> ([a], Int)
froot r = (map (+1) r, length r)

froot [1, 2, 3]

([2,3,4],3)

In [46]:
barOne :: Foldable t => t a -> (t a, Int)
barOne r = (r, length r)

barOne [1, 2, 3]

([1,2,3],3)

In [47]:
barPlus :: (Functor t, Num a, Foldable t) => t a -> (t a, Int)
barPlus r = (foo r, length r)

barPlus [1, 2, 3]

([2,3,4],3)

In [48]:
frooty :: Num a => [a] -> ([a], Int)
frooty r = bar (foo r) r

frooty [1, 2, 3]

([2,3,4],3)

In [49]:
frooty' :: Num a => [a] -> ([a], Int)
frooty' = \r -> bar (foo r) r

frooty' [1, 2, 3]

Line 2: Redundant lambda
Found:
frooty' = \ r -> bar (foo r) r
Why not:
frooty' r = bar (foo r) r

([2,3,4],3)

In [50]:
fooBind :: (r -> a) -> (a -> r -> c) -> (r -> c)
fooBind m k = \r -> k (m r) r

Line 2: Redundant lambda
Found:
fooBind m k = \ r -> k (m r) r
Why not:
fooBind m k r = k (m r) r

---

In [51]:
getDogRM :: Person -> Dog
getDogRM = do
    name <- dogName
    addy <- address
    return $ Dog name addy
    
getDogRM pers
getDogRM chris

Line 2: Use <$>
Found:
do name <- dogName
   addy <- address
   return $ Dog name addy
Why not:
do name <- dogName
   Dog name <$> address

Dog {dogsName = DogName "Barkley", dogsAddress = Address "Sesame Street"}

Dog {dogsName = DogName "Papu", dogsAddress = Address "Austin"}

---

In [52]:
:info Monad

class Applicative m => Monad (m :: * -> *) where
  (>>=) :: m a -> (a -> m b) -> m b
  (>>) :: m a -> m b -> m b
  return :: a -> m a
  fail :: String -> m a
  {-# MINIMAL (>>=) #-}
  	-- Defined in ‘GHC.Base’
instance Monad (Either e) -- Defined in ‘Data.Either’
instance Monad IHaskellTH.Q -- Defined in ‘Language.Haskell.TH.Syntax’
instance Monad [] -- Defined in ‘GHC.Base’
instance Monad Maybe -- Defined in ‘GHC.Base’
instance Monad IO -- Defined in ‘GHC.Base’
instance Monad ((->) r) -- Defined in ‘GHC.Base’
instance Monoid a => Monad ((,) a) -- Defined in ‘GHC.Base’

In [53]:
instance Monad (Reader r) where
    (Reader ra) >>= arb = Reader $ \r -> runReader (arb (ra r)) r

In [54]:
getDogRM' :: Reader Person Dog
getDogRM' = do
    name <- Reader dogName
    addy <- Reader address
    return $ Dog name addy
    
runReader getDogRM' pers
runReader getDogRM' chris

Dog {dogsName = DogName "Barkley", dogsAddress = Address "Sesame Street"}

Dog {dogsName = DogName "Papu", dogsAddress = Address "Austin"}

---

## A warm-up stretch

In [55]:
import Control.Applicative
import Data.Maybe

x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]

In [56]:
:t lookup

lookup :: forall a b. Eq a => a -> [(a, b)] -> Maybe b

In [57]:
xs :: Maybe Integer
xs = lookup 3 (zip x y)
xs

Just 6

In [58]:
ys :: Maybe Integer
ys = lookup 6 (zip y z)
ys

Just 9

In [59]:
zs :: Maybe Integer
zs = lookup 4 (zip x y)
zs

Nothing

In [60]:
z' :: Integer -> Maybe Integer
z' n = lookup n (zip x z)
z' 1

Just 7

In [61]:
x1 :: Maybe (Integer, Integer)
x1 = liftA2 (,) xs ys
x1

Just (6,9)

In [62]:
x2 :: Maybe (Integer, Integer)
x2 = (,) <$> ys <*> zs
x2

Nothing

In [63]:
x3 :: Integer -> (Maybe Integer, Maybe Integer)
x3 = liftA2 (,) z' z'
x3 3

x3' :: Integer -> Maybe (Integer, Integer)
x3' = (liftA2 . liftA2) (,) z' z'
x3' 3

(Just 9,Just 9)

Just (9,9)

In [64]:
:t uncurry

uncurry :: forall a b c. (a -> b -> c) -> (a, b) -> c

In [65]:
summed :: Num c => (c, c) -> c
summed = uncurry (+)

In [66]:
bolt :: Integer -> Bool
bolt = liftA2 (&&) (>3) (<8)

In [67]:
:t fromMaybe

fromMaybe :: forall a. a -> Maybe a -> a

In [68]:
fromMaybe 0 xs

6

In [69]:
fromMaybe 0 zs

0

In [70]:
sequenceA [Just 3, Just 2, Just 1]

Just [3,2,1]

In [71]:
sequenceA [x, y]

[[1,4],[1,5],[1,6],[2,4],[2,5],[2,6],[3,4],[3,5],[3,6]]

In [72]:
sequenceA [xs, ys]

Just [6,9]

In [73]:
summed <$> ((,) <$> xs <*> ys)

Just 15

In [74]:
fmap summed ((,) <$> xs <*> ys)

Just 15

In [75]:
bolt 7

True

In [76]:
fmap bolt z

[True,False,False]

In [77]:
sequenceA [(>3), (<8), even] 7
-- wow this is kinda cool

[True,True,False]

In [78]:
sequA :: Integral a => a -> [Bool]
sequA = sequenceA [(>3), (<8), even]

In [79]:
sequA 7

[True,True,False]

In [80]:
s' = summed <$> ((,) <$> xs <*> ys)
:t s'
s'

s' :: Maybe Integer

Just 15

In [81]:
foldr (&&) True $ sequA 4

Line 1: Use and
Found:
foldr (&&) True
Why not:
and

True

In [82]:
sequA <$> s'

Just [True,False,False]

In [83]:
bolt <$> ys

Just False

## Rewriting Shawty

In [84]:
import Control.Monad.Reader
:info ReaderT

type role ReaderT representational representational nominal
newtype ReaderT r (m :: * -> *) a = ReaderT {runReaderT :: r -> m a}
  	-- Defined in ‘Control.Monad.Trans.Reader’
instance [safe] Functor m => Functor (ReaderT r m) -- Defined in ‘Control.Monad.Trans.Reader’
instance [safe] MonadFix m => MonadFix (ReaderT r m) -- Defined in ‘Control.Monad.Trans.Reader’
instance [safe] Alternative m => Alternative (ReaderT r m) -- Defined in ‘Control.Monad.Trans.Reader’
instance [safe] Applicative m => Applicative (ReaderT r m) -- Defined in ‘Control.Monad.Trans.Reader’
instance [safe] Monad m => Monad (ReaderT r m) -- Defined in ‘Control.Monad.Trans.Reader’
instance [safe] MonadIO m => MonadIO (ReaderT r m) -- Defined in ‘Control.Monad.Trans.Reader’
instance [safe] MonadPlus m => MonadPlus (ReaderT r m) -- Defined in ‘Control.Monad.Trans.Reader’
instance [safe] MonadTrans (ReaderT r) -- Defined in ‘Control.Monad.Trans.Reader’
instance [safe] Monad m => MonadReader r (ReaderT r m) -- Defined in ‘Control.Monad.Reader.Class’

In [85]:
:t replicate

replicate :: forall a. Int -> a -> [a]

In [86]:
listReader :: Reader Int [String]
listReader = do
    content <- ask'
    return $ replicate content "qwer"
    
runReader listReader 3

["qwer","qwer","qwer"]

In [87]:
:t liftIO

liftIO :: forall (m :: * -> *) a. MonadIO m => IO a -> m a

In [88]:
:t putStrLn

putStrLn :: String -> IO ()

In [89]:
:t return :: String -> ReaderT Int [] String

return :: String -> ReaderT Int [] String :: String -> ReaderT Int [] String

In [90]:
:t runReaderT

runReaderT :: forall r (m :: * -> *) a. ReaderT r m a -> r -> m a

In [91]:
:t ask

ask :: forall r (m :: * -> *). MonadReader r m => m r

In [92]:
:info ask

class Monad m => MonadReader r (m :: * -> *) | m -> r where
  ask :: m r
  ...
  	-- Defined in ‘Control.Monad.Reader.Class’

In [93]:
:t (>>=)

(>>=) :: forall (m :: * -> *) a b. Monad m => m a -> (a -> m b) -> m b

In [94]:
:t (>>=) :: ReaderT Int [] String -> (String -> ReaderT Int [] String) -> ReaderT Int [] String

(>>=) :: ReaderT Int [] String -> (String -> ReaderT Int [] String) -> ReaderT Int [] String :: ReaderT Int [] String -> (String -> ReaderT Int [] String) -> ReaderT Int [] String

In [95]:
listReader :: [String] -> [String] -> Reader Int [String]
listReader xs ys = do
    n <- ask'
    return $ do
        x <- xs
        y <- ys
        return $ x ++ concat (replicate n y)

runReader (listReader ["qwer", "asdf"] ["z", "x"]) 3

["qwerzzz","qwerxxx","asdfzzz","asdfxxx"]

In [96]:
asReaderT :: m a -> ReaderT r m a
asReaderT = ReaderT . const

listReaderT :: [String] -> [String] -> ReaderT Int [] String
listReaderT xs ys = do
    n <- ask
    x <- asReaderT xs
    y <- asReaderT ys
    return $ x ++ concat (replicate n y)
    
runReaderT (listReaderT ["qwer", "asdf"] ["z", "x"]) 3

["qwerzzz","qwerxxx","asdfzzz","asdfxxx"]